In [1]:
# from pyfaidx import Fasta
from collections import defaultdict
import pickle
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
%ls

gdrive/  sample_data/


In [4]:
%cd /content/gdrive/MyDrive/seq/

/content/gdrive/MyDrive/seq


## Bed File

In [5]:
sequence = np.genfromtxt("500bp_window_input_200000peaks.bed")

OSError: ignored

In [ ]:
content = []
with open("500bp_window_input_200000peaks.bed")as f:
    for line in f:
        content.append(line.strip().split())

In [ ]:
content

## Extract Sequences

In [ ]:
filepath = './rn6.fa'
genes = Fasta(filepath)

In [7]:
npzfiles = 'peaks/200000peaks-Cck+.npz'
npzfile = np.load(npzfiles)
data, label = npzfile['peaks'], npzfile['values']

In [ ]:
decode_data = []
for d in data:
    decode_data.append(d.decode())

In [8]:
decode_label = []
for l in label:
    decode_label.append(l[0])

In [ ]:
X_seq = []
y_label = []

In [ ]:
for i in range(len(content)):
    tmp = content[i]
    key, start, end = tmp[0], int(tmp[1]), int(tmp[2])
    if key in genes:
        tmpseq = genes[key][start: end].seq
        X_seq.append(tmpseq.upper())
        y_label.append(decode_label[i])

In [ ]:
with open('seq', 'wb') as fp:
    pickle.dump(X_seq, fp)

In [6]:
with open('data/seq', 'rb') as f:
    x = pickle.load(f)

## Encode Sequences

In [15]:
def onehot(X):
    X_onehot = np.zeros((4, len(X)), dtype=np.float32)

    all_cap_letters = ['A', 'T', 'G', 'C']

    for li in range(len(X)):
        letter = X[li]
        X_onehot[all_cap_letters.index(str(letter))][li] = 1

    return X_onehot

In [14]:
one_label = []
one_hot_x = []

In [ ]:
import random

In [ ]:
idxL = random.sample([i for i in range(200000)], 20000)

In [ ]:
idxL = [i for i in range(200000)]

In [10]:
from collections import Counter

In [11]:
Counter(decode_label)[0.0]

155209

In [ ]:
for i, y in enumerate(decode_label):
  if y == 0.0:
    if 'N' in x[i]:
      continue
    one_hot_x.append(onehot(x[i]))
    one_label.append(int(decode_label[i]))
    idxL.remove(i)

In [ ]:
idxR = random.sample(idxL, 15000)

In [16]:
for idx, y in enumerate(decode_label):
    if 'N' in x[idx]:
        continue
    else:
        one_hot_x.append(onehot(x[idx]))
        one_label.append(int(decode_label[idx]))

In [ ]:
one_hot_x = np.array(one_hot_x)
one_label = np.array(one_label)
np.savez('data/ast', one_hot_x, one_label)

In [19]:
cap = 50000

In [18]:
save_x, save_y = [], []

In [20]:
for idx, y in enumerate(one_label):
  if y == 0 and cap > 0:
    save_x.append(one_hot_x[idx])
    save_y.append(0)
    cap -= 1
  else:
    if y == 0: continue
    save_x.append(one_hot_x[idx])
    save_y.append(1)

In [22]:
save_x = np.array(save_x)
save_y = np.array(save_y)
np.savez('data/cck_b', save_x, save_y)

In [23]:
save_x

array([[[0., 0., 1., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 1., 0., 1.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 1., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 1., 0.]],

       [[1., 1., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]],

       [[0., 1., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 1.]],

       [[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 1., 1., 0.]]], dtype=float32)